In [1]:
#@title Mount google drive
from google.colab import drive
drive.mount('/content/drive')
from sys import version_info
python_version = f"{version_info.major}.{version_info.minor}"

Mounted at /content/drive


In [7]:
#@title Input protein sequence, then hit `Runtime` -> `Run all`

input_dir = '/content/drive/MyDrive/alphafold2/unfiltered_rest_set.fasta' #@param {type:"string"}
result_dir = '/content/drive/MyDrive/alphafold2/result' #@param {type:"string"}

# number of models to use
#@markdown ---
#@markdown ### Advanced settings
msa_mode = "MMseqs2 (UniRef+Environmental)" #@param ["MMseqs2 (UniRef+Environmental)", "MMseqs2 (UniRef only)","single_sequence","custom"]
num_models = 5 #@param [1,2,3,4,5] {type:"raw"}
num_recycles = 3 #@param [1,3,6,12,24,48] {type:"raw"}
stop_at_score = 100 #@param {type:"string"}
#@markdown - early stop computing models once score > threshold (avg. plddt for "structures" and ptmscore for "complexes")
use_custom_msa = False
num_relax = 0 #@param [0, 1, 5] {type:"raw"}
use_amber = num_relax > 0
relax_max_iterations = 200 #@param [0,200,2000] {type:"raw"}
use_templates = False #@param {type:"boolean"}
do_not_overwrite_results = True #@param {type:"boolean"}
zip_results = False #@param {type:"boolean"}

In [8]:
#@title Install dependencies
%%bash -s $use_amber $use_templates $python_version

set -e

USE_AMBER=$1
USE_TEMPLATES=$2
PYTHON_VERSION=$3

if [ ! -f COLABFOLD_READY ]; then
  # install dependencies
  # We have to use "--no-warn-conflicts" because colab already has a lot preinstalled with requirements different to ours
  pip install -q --no-warn-conflicts "colabfold[alphafold-minus-jax] @ git+https://github.com/sokrypton/ColabFold"
  if [ -n "${TPU_NAME}" ]; then
    pip install -q --no-warn-conflicts -U dm-haiku==0.0.10 jax==0.3.25
  fi
  ln -s /usr/local/lib/python3.*/dist-packages/colabfold colabfold
  ln -s /usr/local/lib/python3.*/dist-packages/alphafold alphafold
  touch COLABFOLD_READY
fi

# Download params (~1min)
python -m colabfold.download

# setup conda
if [ ${USE_AMBER} == "True" ] || [ ${USE_TEMPLATES} == "True" ]; then
  if [ ! -f CONDA_READY ]; then
    wget -qnc https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
    bash Miniconda3-latest-Linux-x86_64.sh -bfp /usr/local 2>&1 1>/dev/null
    rm Miniconda3-latest-Linux-x86_64.sh
    conda config --set auto_update_conda false
    touch CONDA_READY
  fi
fi
# setup template search
if [ ${USE_TEMPLATES} == "True" ] && [ ! -f HH_READY ]; then
  conda install -y -q -c conda-forge -c bioconda kalign2=2.04 hhsuite=3.3.0 python="${PYTHON_VERSION}" 2>&1 1>/dev/null
  touch HH_READY
fi
# setup openmm for amber refinement
if [ ${USE_AMBER} == "True" ] && [ ! -f AMBER_READY ]; then
  conda install -y -q -c conda-forge openmm=8.2.0 python="${PYTHON_VERSION}" pdbfixer 2>&1 1>/dev/null
  touch AMBER_READY
fi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.2/242.2 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.9/373.9 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.8/251.8 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 122.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 121.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 kB 7.3 MB/s eta 0:00:00


In [9]:
#@title Run Prediction

import sys

from colabfold.batch import get_queries, run
from colabfold.download import default_data_dir
from colabfold.utils import setup_logging
from pathlib import Path

# For some reason we need that to get pdbfixer to import
if use_amber and f"/usr/local/lib/python{python_version}/site-packages/" not in sys.path:
    sys.path.insert(0, f"/usr/local/lib/python{python_version}/site-packages/")

setup_logging(Path(result_dir).joinpath("log.txt"))

queries, is_complex = get_queries(input_dir)
run(
    queries=queries,
    result_dir=result_dir,
    use_templates=use_templates,
    num_relax=num_relax,
    relax_max_iterations=relax_max_iterations,
    msa_mode=msa_mode,
    model_type="auto",
    num_models=num_models,
    num_recycles=num_recycles,
    model_order=[1, 2, 3, 4, 5],
    is_complex=is_complex,
    data_dir=default_data_dir,
    keep_existing_results=do_not_overwrite_results,
    rank_by="auto",
    pair_mode="unpaired+paired",
    stop_at_score=stop_at_score,
    zip_results=zip_results,
    user_agent="colabfold/google-colab-batch",
)

2025-05-11 13:30:51,940 Running on GPU
2025-05-11 13:30:52,302 Found 5 citations for tools or databases
2025-05-11 13:30:52,303 Query 1/3: 30206_1_2_2 (length 12)


PENDING:   0%|          | 0/150 [elapsed: 00:00 remaining: ?]

2025-05-11 13:30:52,553 Sleeping for 9s. Reason: PENDING


COMPLETE: 100%|██████████| 150/150 [elapsed: 00:09 remaining: 00:00]


2025-05-11 13:31:14,607 Padding length to 22
2025-05-11 13:31:37,095 alphafold2_ptm_model_1_seed_000 recycle=0 pLDDT=75.8 pTM=0.0446
2025-05-11 13:31:40,062 alphafold2_ptm_model_1_seed_000 recycle=1 pLDDT=80.2 pTM=0.0469 tol=0.423
2025-05-11 13:31:43,027 alphafold2_ptm_model_1_seed_000 recycle=2 pLDDT=79.1 pTM=0.0466 tol=0.237
2025-05-11 13:31:45,993 alphafold2_ptm_model_1_seed_000 recycle=3 pLDDT=77.5 pTM=0.0446 tol=0.333
2025-05-11 13:31:45,993 alphafold2_ptm_model_1_seed_000 took 31.4s (3 recycles)
2025-05-11 13:31:48,974 alphafold2_ptm_model_2_seed_000 recycle=0 pLDDT=69.1 pTM=0.0438
2025-05-11 13:31:51,936 alphafold2_ptm_model_2_seed_000 recycle=1 pLDDT=74.5 pTM=0.0464 tol=0.948
2025-05-11 13:31:54,894 alphafold2_ptm_model_2_seed_000 recycle=2 pLDDT=71.2 pTM=0.0443 tol=0.205
2025-05-11 13:31:57,847 alphafold2_ptm_model_2_seed_000 recycle=3 pLDDT=72.1 pTM=0.0439 tol=0.41
2025-05-11 13:31:57,847 alphafold2_ptm_model_2_seed_000 took 11.8s (3 recycles)
2025-05-11 13:32:00,813 alphafol

COMPLETE: 100%|██████████| 150/150 [elapsed: 00:00 remaining: 00:00]


2025-05-11 13:32:36,631 Padding length to 54
2025-05-11 13:33:01,466 alphafold2_ptm_model_1_seed_000 recycle=0 pLDDT=62.8 pTM=0.305
2025-05-11 13:33:05,971 alphafold2_ptm_model_1_seed_000 recycle=1 pLDDT=62.4 pTM=0.308 tol=0.664
2025-05-11 13:33:10,490 alphafold2_ptm_model_1_seed_000 recycle=2 pLDDT=64.2 pTM=0.331 tol=0.487
2025-05-11 13:33:15,036 alphafold2_ptm_model_1_seed_000 recycle=3 pLDDT=63.6 pTM=0.325 tol=0.151
2025-05-11 13:33:15,037 alphafold2_ptm_model_1_seed_000 took 38.4s (3 recycles)
2025-05-11 13:33:19,604 alphafold2_ptm_model_2_seed_000 recycle=0 pLDDT=53.3 pTM=0.26
2025-05-11 13:33:24,178 alphafold2_ptm_model_2_seed_000 recycle=1 pLDDT=56.2 pTM=0.283 tol=0.435
2025-05-11 13:33:28,776 alphafold2_ptm_model_2_seed_000 recycle=2 pLDDT=56.8 pTM=0.298 tol=0.232
2025-05-11 13:33:33,377 alphafold2_ptm_model_2_seed_000 recycle=3 pLDDT=55.5 pTM=0.287 tol=0.76
2025-05-11 13:33:33,378 alphafold2_ptm_model_2_seed_000 took 18.3s (3 recycles)
2025-05-11 13:33:38,022 alphafold2_ptm_mo

COMPLETE: 100%|██████████| 150/150 [elapsed: 00:00 remaining: 00:00]


2025-05-11 13:35:01,965 alphafold2_ptm_model_1_seed_000 recycle=0 pLDDT=85.8 pTM=0.701
2025-05-11 13:35:08,418 alphafold2_ptm_model_1_seed_000 recycle=1 pLDDT=83 pTM=0.668 tol=0.448
2025-05-11 13:35:14,926 alphafold2_ptm_model_1_seed_000 recycle=2 pLDDT=83.4 pTM=0.666 tol=0.608
2025-05-11 13:35:21,493 alphafold2_ptm_model_1_seed_000 recycle=3 pLDDT=80.9 pTM=0.632 tol=0.684
2025-05-11 13:35:21,494 alphafold2_ptm_model_1_seed_000 took 47.9s (3 recycles)
2025-05-11 13:35:28,141 alphafold2_ptm_model_2_seed_000 recycle=0 pLDDT=85.9 pTM=0.704
2025-05-11 13:35:34,778 alphafold2_ptm_model_2_seed_000 recycle=1 pLDDT=80.8 pTM=0.635 tol=0.567
2025-05-11 13:35:41,364 alphafold2_ptm_model_2_seed_000 recycle=2 pLDDT=80.8 pTM=0.636 tol=0.408
2025-05-11 13:35:47,903 alphafold2_ptm_model_2_seed_000 recycle=3 pLDDT=80.3 pTM=0.627 tol=0.279
2025-05-11 13:35:47,904 alphafold2_ptm_model_2_seed_000 took 26.4s (3 recycles)
2025-05-11 13:35:54,435 alphafold2_ptm_model_3_seed_000 recycle=0 pLDDT=85.9 pTM=0.71


{'rank': [['rank_001_alphafold2_ptm_model_5_seed_000',
   'rank_002_alphafold2_ptm_model_4_seed_000',
   'rank_003_alphafold2_ptm_model_1_seed_000',
   'rank_004_alphafold2_ptm_model_3_seed_000',
   'rank_005_alphafold2_ptm_model_2_seed_000'],
  ['rank_001_alphafold2_ptm_model_1_seed_000',
   'rank_002_alphafold2_ptm_model_3_seed_000',
   'rank_003_alphafold2_ptm_model_5_seed_000',
   'rank_004_alphafold2_ptm_model_2_seed_000',
   'rank_005_alphafold2_ptm_model_4_seed_000'],
  ['rank_001_alphafold2_ptm_model_3_seed_000',
   'rank_002_alphafold2_ptm_model_5_seed_000',
   'rank_003_alphafold2_ptm_model_4_seed_000',
   'rank_004_alphafold2_ptm_model_1_seed_000',
   'rank_005_alphafold2_ptm_model_2_seed_000']],
 'metric': [[{'mean_plddt': 80.875,
    'ptm': 0.046478271484375,
    'print_line': ' pLDDT=80.9 pTM=0.0465'},
   {'mean_plddt': 80.375,
    'ptm': 0.0457763671875,
    'print_line': ' pLDDT=80.4 pTM=0.0458'},
   {'mean_plddt': 77.5,
    'ptm': 0.044647216796875,
    'print_line': '